# Team Analizer during draft

## set up

In [1]:
import numpy as np
import pandas as pd

from sleeper_wrapper import League
import requests


In [2]:
league = League(692104824862367744)

In [3]:
data_in = './data_in/'
data_out = './data_out/'
data_postfix = 'dynasty-ppr-rankings'

In [4]:
ALL = pd.read_csv('{}All-{}.csv'.format(data_in,data_postfix))

## add initial picks with 3rd round reverse

In [5]:
fwards=[0,1,2,3,4,5,6,7,8,9,10,11]
bwards=[11,10,9,8,7,6,5,4,3,2,1,0]

In [6]:
picks={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[]}

In [7]:
players={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[]}
players_val={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[]}

In [9]:
for k,v in enumerate(fwards):
    picks[k+1].append(v)
for k,v in enumerate(bwards):
    picks[k+1].append(v+12)
for k,v in enumerate(bwards):
    picks[k+1].append(v+24)

In [10]:
for x in range(4,30,2):
    for k,v in enumerate(fwards):
        picks[k+1].append(v+(x-1)*12)
    for k,v in enumerate(bwards):
        picks[k+1].append(v+(x)*12)

### initial pick value

In [11]:
#sums up the values of the picks, based off the lifetime value of optimal picks
for k in picks:
    print(ALL.iloc[picks[k]]['Lifetime Value'].values.sum())

1879.8
1839.8200000000002
1841.87
1779.81
1785.4299999999996
1766.97
1770.07
1759.99
1761.01
1759.83
1766.1000000000001
1773.4500000000003


### roster 2 slot

In [12]:
draft_meta=requests.get("https://api.sleeper.app/v1/draft/692104825852223488")

In [14]:
roster_id_slots=draft_meta.json()['slot_to_roster_id']
roster_id_slots = {v:int(k) for k, v in roster_id_slots.items()}
roster_id_slots

{12: 9, 7: 8, 10: 7, 2: 6, 9: 5, 5: 4, 4: 3, 8: 2, 1: 12, 6: 11, 3: 10, 11: 1}

In [15]:
def roster_2_slot(x):
    return roster_id_slots[x]

In [16]:
def slot_2_roster(x):
    slot_2_roster = {v:int(k) for k, v in roster_id_slots.items()}
    return slot_2_roster[x]

## scrape the traded picks so far

In [18]:
traded=requests.get("https://api.sleeper.app/v1/draft/692104825852223488/traded_picks")

In [19]:
traded.json()

[{'season': '2021',
  'round': 2,
  'roster_id': 1,
  'previous_owner_id': 1,
  'owner_id': 12},
 {'season': '2021',
  'round': 3,
  'roster_id': 1,
  'previous_owner_id': 1,
  'owner_id': 4},
 {'season': '2021',
  'round': 4,
  'roster_id': 1,
  'previous_owner_id': 1,
  'owner_id': 10},
 {'season': '2021',
  'round': 12,
  'roster_id': 1,
  'previous_owner_id': 1,
  'owner_id': 4},
 {'season': '2021',
  'round': 1,
  'roster_id': 2,
  'previous_owner_id': 2,
  'owner_id': 11},
 {'season': '2021',
  'round': 3,
  'roster_id': 2,
  'previous_owner_id': 2,
  'owner_id': 11},
 {'season': '2021',
  'round': 3,
  'roster_id': 3,
  'previous_owner_id': 6,
  'owner_id': 7},
 {'season': '2021',
  'round': 4,
  'roster_id': 3,
  'previous_owner_id': 8,
  'owner_id': 9},
 {'season': '2021',
  'round': 2,
  'roster_id': 4,
  'previous_owner_id': 6,
  'owner_id': 5},
 {'season': '2021',
  'round': 3,
  'roster_id': 4,
  'previous_owner_id': 4,
  'owner_id': 1},
 {'season': '2021',
  'round': 4,
 

In [20]:
trade_table=pd.json_normalize(traded.json())
trade_table=trade_table.rename(columns={'round':'Rou'})

### assign the pick number to the traded pick

In [21]:
trade_table['pick']=trade_table.apply(lambda x: picks[roster_id_slots[x.roster_id]][x.Rou-1]+1,axis=1)

In [22]:
trade_table.sort_values('pick').head(50)

,season,Rou,roster_id,previous_owner_id,owner_id,pick
56,2021,1,11,11,2,1
38,2021,1,8,8,9,2
48,2021,1,9,9,8,5
4,2021,1,2,2,11,6
31,2021,1,7,7,5,8
22,2021,1,6,6,4,11
0,2021,2,1,1,12,13
61,2021,2,12,12,8,16
32,2021,2,7,7,8,17
49,2021,2,9,8,12,20


### remove the pick from the old team, and give it to the new team

In [23]:
for row in trade_table.itertuples():
    picks[roster_id_slots[row.roster_id]].remove(row.pick-1)
    picks[roster_id_slots[row.owner_id]].append(row.pick-1)

In [1]:
#dictionary to fix name discrepencies
name_dict={
        'DK Metcalf':"D.K. Metcalf"
}

## Scrape the draft pick results

In [25]:
draft=requests.get("https://api.sleeper.app/v1/draft/692104825852223488/picks")

In [26]:
draft.json()

[{'round': 1,
  'roster_id': 2,
  'player_id': '4046',
  'picked_by': '464605759842283520',
  'pick_no': 1,
  'metadata': {'years_exp': '4',
   'team': 'KC',
   'status': 'Active',
   'sport': 'nfl',
   'position': 'QB',
   'player_id': '4046',
   'number': '15',
   'news_updated': '1618848913690',
   'last_name': 'Mahomes',
   'injury_status': 'Questionable',
   'first_name': 'Patrick'},
  'is_keeper': None,
  'draft_slot': 1,
  'draft_id': '692104825852223488'},
 {'round': 1,
  'roster_id': 9,
  'player_id': '4984',
  'picked_by': '693183322456686592',
  'pick_no': 2,
  'metadata': {'years_exp': '3',
   'team': 'BUF',
   'status': 'Active',
   'sport': 'nfl',
   'position': 'QB',
   'player_id': '4984',
   'number': '17',
   'news_updated': '1620074736656',
   'last_name': 'Allen',
   'injury_status': '',
   'first_name': 'Josh'},
  'is_keeper': None,
  'draft_slot': 2,
  'draft_id': '692104825852223488'},
 {'round': 1,
  'roster_id': 4,
  'player_id': '5849',
  'picked_by': '6947641

In [27]:
draft_df=pd.json_normalize(draft.json())

In [29]:
draft_df['player_name']=draft_df['metadata.first_name'] + ' ' + draft_df['metadata.last_name']

In [30]:
draft_df['player_name']=draft_df['player_name'].replace(name_dict)

### remove the pick and add the player to the teams roster

In [32]:
for row in draft_df.itertuples():
    x=roster_2_slot(row.roster_id)
    pick=row.pick_no-1
    picks[x].remove(pick)
    players[x].append(row.player_name)

In [33]:
players

{1: ['Saquon Barkley',
  'Trey Lance',
  'Justin Fields',
  'George Kittle',
  'Najee Harris'],
 2: ['Christian McCaffrey', 'Tyreek Hill', 'Nick Chubb', 'DeAndre Hopkins'],
 3: ['Kyler Murray', 'Joe Burrow', 'D.K. Metcalf', 'Travis Etienne'],
 4: ['Justin Herbert', 'Lamar Jackson', 'Davante Adams'],
 5: ['Josh Allen',
  'A.J. Brown',
  'Aaron Jones',
  'Stefon Diggs',
  'Aaron Rodgers'],
 6: ['Patrick Mahomes', 'Ezekiel Elliott', 'Darren Waller'],
 7: ['Dalvin Cook', 'Derrick Henry', 'Travis Kelce', 'Ryan Tannehill'],
 8: ['Justin Jefferson',
  'Clyde Edwards-Helaire',
  'Zach Wilson',
  'Austin Ekeler'],
 9: ['Jonathan Taylor', 'Alvin Kamara', 'Russell Wilson', 'Calvin Ridley'],
 10: ['Dak Prescott',
  'Cam Akers',
  'J.K. Dobbins',
  'Tua Tagovailoa',
  'Miles Sanders'],
 11: ["D'Andre Swift",
  'Joe Mixon',
  'Jalen Hurts',
  'Antonio Gibson',
  'Deshaun Watson'],
 12: ['Trevor Lawrence',
  'Kyle Pitts',
  "Ja'Marr Chase",
  'CeeDee Lamb',
  'Javonte Williams']}

## Get the value of all the players

In [34]:
for key in players:
    for values in players[key]:
        x=ALL[ALL['Full Name']==values]['Lifetime Value'].values[0]
        players_val[key].append(x)

In [35]:
players_val

{1: [260.91, 200.92, 195.12, 138.18, 178.18],
 2: [265.99, 150.86, 156.68, 119.04],
 3: [353.21, 183.2, 167.86, 184.2],
 4: [247.64, 208.79, 164.94],
 5: [285.71, 209.52, 145.04, 146.67, 158.88],
 6: [394.45, 159.71, 143.27],
 7: [204.76, 146.94, 156.65, 174.49],
 8: [175.89, 138.6, 140.4, 124.03],
 9: [287.26, 188.47, 241.03, 135.61],
 10: [354.29, 245.68, 228.54, 160.68, 148.75],
 11: [240.21, 160.07, 232.56, 190.55, 190.34],
 12: [214.8, 126.75, 138.99, 170.1, 134.85]}

## Team Values = player value + optimal pick value

In [36]:
for k in picks:
    p=ALL.iloc[picks[k]]['Lifetime Value'].values.sum().round(2)
    pla=np.array(players_val[k]).sum().round(2)
    print(f'User {slot_2_roster(k)} has {pla} value in {len(players_val[k])} players\nand {p} value in {len(picks[k])} picks left\nfor a total of {round(p+pla,2)}')
    print(f'from draft slot {k}\n\n')

User 1 has 973.31 value in 5 players
and 967.25 value in 25 picks left
for a total of 1940.56
from draft slot 11


User 2 has 692.57 value in 4 players
and 924.78 value in 24 picks left
for a total of 1617.35
from draft slot 8


User 3 has 888.47 value in 4 players
and 805.69 value in 24 picks left
for a total of 1694.16
from draft slot 4


User 4 has 621.37 value in 3 players
and 1031.87 value in 25 picks left
for a total of 1653.24
from draft slot 5


User 5 has 945.82 value in 5 players
and 848.85 value in 24 picks left
for a total of 1794.67
from draft slot 9


User 6 has 697.43 value in 3 players
and 1000.14 value in 25 picks left
for a total of 1697.57
from draft slot 2


User 7 has 682.84 value in 4 players
and 1040.9 value in 25 picks left
for a total of 1723.74
from draft slot 10


User 8 has 578.92 value in 4 players
and 1252.94 value in 28 picks left
for a total of 1831.86
from draft slot 7


User 9 has 852.37 value in 4 players
and 884.08 value in 24 picks left
for a total 